<a href="https://colab.research.google.com/github/cafs92/FeatureSelectionPython/blob/master/Pipeline_de_Feature_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports e filtro de dataset


In [184]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold,  SelectKBest, f_classif
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [185]:
def top_features(mask, feature_names):
  new_features = [] # The list of your K best features

  for bool, feature in zip(mask, feature_names):
      if bool:
          new_features.append(feature)
  return {'Selected Features': new_features}

In [186]:
ds = pd.read_csv('/content/Chagas - Dataset Novo - ufrj-revisada.csv')
ds_processed = ds.drop([  'Filename','Data MS','DIAS HOLTER-MS','Paciente', 'Unnamed: 7',
                          'Nome do Paciente','Prontuario','Date Nasc ','Data Holter','Nat.', 'Cardiopatia Outra', 
                          'Embolia Pulmonar', 'ECG ', 'FC', 'Holter', 'ECO', 'Obito', 'OBSERVACOES', 'STATUS_OBITO',
                          'CHECK Nome', 'Nome do Paciente Excell', 'CHECK MS', 'Morte Súbita Excell', 'Data MS Excell',
                          'OBS Excell'
                          ], axis = 1)
ds_clean_status = ds_processed['STATUS'] == "COMPLETO" #Filtrar pra pegar dados clínicos + holter
ds_clean_last_holter = ds_processed['Last Holter'] == 1
ds_models = ds_processed[ds_clean_status & ds_clean_last_holter]
ds_models = ds_models.drop(['STATUS','Last Holter'], axis = 1)
ds_models[ds_models.isna().any(axis=1)] 
ds_models = ds_models.dropna(axis=0, how='any')

ds_models.shape

(217, 58)

In [187]:
#One Hot Encoding
aux = pd.get_dummies(ds_models[['Classificação', 'Rassi escore', 'Diretriz 2005']])
for i in aux.columns:
  ds_models[i] = aux[i]

ds_models = ds_models.drop(['Classificação', 'Rassi escore', 'Diretriz 2005'], axis = 1)
ds_models.shape

(217, 67)

In [188]:
train_features, test_features, train_labels, test_labels=train_test_split(
    ds_models.drop(labels=['MS'], axis=1),
    ds_models['MS'],
    test_size=0.2,
    random_state=42)

# Constant Feature Removal

In [189]:
constant_filter = VarianceThreshold(threshold=0)
constant_filter.fit(train_features)
len(train_features.columns[constant_filter.get_support()])

64

In [190]:
constant_columns = [column for column in train_features.columns
                    if column not in train_features.columns[constant_filter.get_support()]]
constant_columns

['Last Exam', 'TVS_H']

In [191]:
train_features = constant_filter.transform(train_features)
test_features = constant_filter.transform(test_features)

train_features.shape, test_features.shape

((173, 64), (44, 64))

In [192]:
ds_models = ds_models.drop(constant_columns, axis = 1)

#Quasi-Constant Feature Removal

In [193]:
train_features, test_features, train_labels, test_labels=train_test_split(
    ds_models.drop(labels=['MS'], axis=1),
    ds_models['MS'],
    test_size=0.2,
    random_state=42)

In [194]:
qconstant_filter = VarianceThreshold(threshold=0.01)
qconstant_filter.fit(train_features)
len(train_features.columns[qconstant_filter.get_support()])

62

In [195]:
qconstant_columns = [column for column in train_features.columns
                    if column not in train_features.columns[qconstant_filter.get_support()]]
qconstant_columns

['Alcoolismo', 'Rassi escore_Baixo']

In [196]:
train_features = qconstant_filter.transform(train_features)
test_features = qconstant_filter.transform(test_features)

train_features.shape, test_features.shape

((173, 62), (44, 62))

In [197]:
ds_models = ds_models.drop(qconstant_columns, axis = 1)

# Redundant Feature Removal

In [198]:
train_features, test_features, train_labels, test_labels=train_test_split(
    ds_models.drop(labels=['MS'], axis=1),
    ds_models['MS'],
    test_size=0.2,
    random_state=42)

In [199]:
train_features_T = train_features.T
train_features_T.shape

(62, 173)

In [200]:
train_features_T.duplicated().sum()

0

In [201]:
unique_features = train_features_T.drop_duplicates(keep='first').T
unique_features.shape

(173, 62)

In [202]:
duplicated_features = [dup_col for dup_col in train_features.columns if dup_col not in unique_features.columns]
duplicated_features

[]

In [203]:
ds_models = ds_models.drop(duplicated_features, axis = 1)

# Correlated Features Removal

In [204]:
num_colums = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_columns = list(ds_models.select_dtypes(include=num_colums).columns)
ds_models2 = ds_models[numerical_columns]
ds_models2.shape

(217, 52)

Interessante ser o primeiro uma vez que o cara no tutorial faz isso de novo lá

In [205]:
train_features2, test_features2, train_labels2, test_labels2 = train_test_split(
    ds_models2.drop(labels=['MS'], axis=1),
    ds_models2['MS'],
    test_size=0.2,
    random_state=42)

In [206]:
correlated_features = set()
correlation_matrix = ds_models2.corr()

In [207]:
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)
correlated_features

{'Disf Diastolica',
 'Dist Cond AtrioVent ',
 'EVTotal_H',
 'FE Teicholz',
 'Rassi pontos',
 'TVMNS_H',
 'TpeakQ 5th Percentile (s)',
 'VES'}

In [208]:
train_features2.drop(labels=correlated_features, axis=1, inplace=True)
test_features2.drop(labels=correlated_features, axis=1, inplace=True)

train_features2.shape

(173, 43)

In [209]:
ds_models = ds_models.drop(correlated_features, axis = 1)

In [210]:
ds_models.columns

Index(['Sexo', 'BMI ', 'Cancer', 'HAS', 'DM2', 'Marcapasso', 'Sincope',
       'Fibrilação/Flutter Atrial', 'I R Crônica', 'DLP', 'Coronariopatia',
       'Ins Cardiaca ', 'AVC', 'DVP', 'TSH', 'Tabagismo', 'Sedentarismo',
       'Alt Prim', 'Dist Cond InterVent ', 'Pausa > 3s ', 'ESV', 'EV', 'TVMNS',
       'Area Elet inativa', 'Dist Cond AtrioVent_H', 'Disf Nodulo Sinusal_H',
       'Fibri/Flutter Atrial_H', 'FC media_H', 'EV_H', 'AE diam.', 'VED',
       'Deficit Seg', 'NYHA', 'CDI ', 'Ablações', 'Amiodarona', 'MS',
       'Idade no exame', 'TendQ 5th Percentile (s)',
       'Percentage (%) of beats with QTend/TendQ > 1',
       'Percentage (%) of beats with QTpeak/TpeakQ > 1',
       'Upper 98% quantil of the QTend/TendQ ratio',
       'Upper 98% quantil of the QTpeak/TpeakQ ratio',
       'Tpeak-Tend 5th-Percentile (ms)', 'Classificação_Disf Grave',
       'Classificação_Disf Leve', 'Classificação_Disf Moderada',
       'Classificação_Normal', 'Rassi escore_alto', 'Rassi escore_bai

# ANOVA

In [211]:
constant_columns, qconstant_columns, duplicated_features, correlated_features

(['Last Exam', 'TVS_H'],
 ['Alcoolismo', 'Rassi escore_Baixo'],
 [],
 {'Disf Diastolica',
  'Dist Cond AtrioVent ',
  'EVTotal_H',
  'FE Teicholz',
  'Rassi pontos',
  'TVMNS_H',
  'TpeakQ 5th Percentile (s)',
  'VES'})

In [212]:
train_features, test_features, train_labels, test_labels=train_test_split(
    ds_models.drop(labels=['MS'], axis=1),
    ds_models['MS'],
    test_size=0.2,
    random_state=42)

In [213]:
anova = SelectKBest(score_func=f_classif, k=25)
X_selected_anova = anova.fit_transform(train_features, train_labels)
selected_anova = top_features(anova.get_support(), ds_models.columns)

selected_anova

{'Selected Features': ['HAS',
  'Sincope',
  'DVP',
  'Alt Prim',
  'ESV',
  'EV',
  'TVMNS',
  'FC media_H',
  'EV_H',
  'AE diam.',
  'VED',
  'Deficit Seg',
  'NYHA',
  'Amiodarona',
  'MS',
  'Percentage (%) of beats with QTend/TendQ > 1',
  'Upper 98% quantil of the QTpeak/TpeakQ ratio',
  'Classificação_Disf Leve',
  'Classificação_Disf Moderada',
  'Classificação_Normal',
  'Rassi escore_alto',
  'Rassi escore_baixo',
  'Rassi escore_intermediario',
  'Diretriz 2005_A',
  'Diretriz 2005_B1']}